# Building steel coils

This tutorial includes everything you need to set up decision optimization engines, and build constraint programming models to optimize the steel mill slab design problem solution. This problem is based on "prob038: Steel mill slab design problem" from [CSPLib](http://www.csplib.org). All data and instructions that you need to model and solve this problem are included.


When you finish this tutorial, you'll have a foundational knowledge of _Prescriptive Analytics_.

>This notebook is part of [Prescriptive Analytics for Python](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html).

>This notebook requires a valid subscription to [Decision Optimization on Cloud](https://developer.ibm.com/docloud) or a local installation of CPLEX Optimizers.

Some familiarity with Python is recommended.


 ## Table of contents 

-  [Describe the business problem](#Describe-the-business-problem)<br>
-  [How decision optimization (prescriptive analytics) can help](#How--decision-optimization-can-help)<br>
-  [Use decision optimization](#Use-decision-optimization)<br>
    1.  [Download the library](#Download-library)<br>
    2.  [Set up the engines](#Step-2:-Set-up-the-prescriptive-engine)<br>
    3.  [Model the data](#Step-3:-Model-the-data)<br>
    4.  [Set up the prescriptive model](#Step-4:-Set-up-the-prescriptive-model)<br>
        4.1 [Define the decision variables](#Define-the-decision-variables)<br>
        4.2 [Express the business constraints](#Express-the-business-constraints)<br>
        4.3 [Express the objective](#Express-objective)<br>
        4.4 [Solve with Decision Optimization solve service](#Solve-with-Decision-Optimization-solve-service)<br>
    5. [Investigate the solution and run an example analysis](#Step-5:-Investigate-the-solution-and-then-run-an-example-analysis)<br>
-  [Summary](#Summary)<br>
****
   

### Describe the business problem

* The problem is to build steel coils from slabs that are available in a work-in-process inventory of semi-finished products. There is no limitation in the number of slabs that can be requested, but only a finite number of slab sizes is available (sizes 11, 13, 16, 17, 19, 20, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 40, 43, 45). 
* The problem is to select a number of slabs to build the coil orders, and to satisfy the following constraints:
    * A coil order can be built from only one slab.
    * Each coil order requires a specific process to build it from a slab. This process is encoded by a color.
    * Several coil orders can be built from the same slab. But a slab can be used to produce at most two different "colors" of coils.
    * The sum of the sizes of each coil order built from a slab must not exceed the slab size.


* Finally, the production plan should minimize the unused capacity of the selected slabs.

## How  decision optimization can help

Prescriptive analytics (decision optimization) technology recommends actions that are based on desired outcomes. It considers specific scenarios, resources, and knowledge of past and current events. With this insight, your organization can make better decisions and have greater control over business outcomes.

Prescriptive analytics is the next step on the path to insight-based actions. It creates value through synergy with predictive analytics, which analyzes data to predict future outcomes. Prescriptive analytics takes that insight to the next level by suggesting the optimal way to handle a future situation. Organizations that act fast in dynamic conditions and make superior decisions in uncertain environments gain a strong competitive advantage.

With prescriptive analytics, you can:

* Automate the complex decisions and trade-offs to better manage your limited resources.
    
* Take advantage of a future opportunity or mitigate a future risk.
    
* Proactively update recommendations based on changing events.
    
* Meet operational goals, increase customer loyalty, prevent threats and fraud, and optimize business processes.


## Use decision optimization

<a id="Download-library"></a>
## 1. Download the DOcplex library

DOcplex is the Decision Optimization CPLEX Modeling library for Python. This library includes two modules, Mathematical Programming Modeling (DOcplex.MP ) and Constraint Programming Modeling (DOcplex.CP).

Run the following cell to install the library. The `real.prefix` in the code is used to detect whether the script is running inside a virtual environment, and adjust the install procedure accordingly.

In [1]:
!pip install docplex

In [2]:
import docplex.cp

Note that the more global package <i>docplex</i> contains another subpackage <i>docplex.mp</i> that is dedicated to Mathematical Programming, another branch of optimization.

<a id="Step-2:-Set-up-the-prescriptive-engine"></a>
## 2. Set up the prescriptive engine

To access the DOcplexcloud solve service, perform the following steps:

* Subscribe to the [Decision Optimization on Cloud](https://developer.ibm.com/docloud) (DOcplexcloud) service.
* Get the service base URL and personal API key.
* Enter the URL and API key in the cell below, enclosed in quotation marks (""), and run the cell. 

__Note:__ For a persistent setting, create a Python configuration file *docloud_config.py* in a location that is visible to PYTHONPATH.

In [3]:
# Initialize IBM Decision Optimization credentials
SVC_URL = "ENTER YOUR URL HERE" 
SVC_KEY = "ENTER YOUR CREDENTIAL ID HERE"

Import the DOcplex library. 

In [4]:
from docplex.cp.model import *

<div class="alert alert-block alert-info"> Note: This notebook requires a full subscription to CPLEX 12.7.1 or above, that is with a valid docplexcloud url and key.</div> 

<a id="Step-3:-Model-the-data"></a>
## 3. Model the data

Set the model parameters. The comments in the code provide details about the data.

In [5]:
from collections import namedtuple

##############################################################################
# Model configuration
##############################################################################

# The number of coils to produce
TUPLE_ORDER = namedtuple("TUPLE_ORDER", ["index", "weight", "color"])
orders = [ TUPLE_ORDER(1, 22, 5),
           TUPLE_ORDER(2,  9, 3),
           TUPLE_ORDER(3,  9, 4),
           TUPLE_ORDER(4,  8, 5),
           TUPLE_ORDER(5,  8, 7),
           TUPLE_ORDER(6,  6, 3),
           TUPLE_ORDER(7,  5, 6),
           TUPLE_ORDER(8,  3, 0),
           TUPLE_ORDER(9,  3, 2),
           TUPLE_ORDER(10, 3, 3),
           TUPLE_ORDER(11, 2, 1),
           TUPLE_ORDER(12, 2, 5)
           ]

NB_SLABS = 12
MAX_COLOR_PER_SLAB = 2

# The total number of slabs available.  In theory this can be unlimited,
# but we impose a reasonable upper bound in order to produce a practical
# optimization model.

# The different slab weights available.
slab_weights = [ 0, 11, 13, 16, 17, 19, 20, 23, 24, 25,
                 26, 27, 28, 29, 30, 33, 34, 40, 43, 45 ]

In [6]:
nb_orders = len(orders)
slabs = range(NB_SLABS)
allcolors = set([ o.color for o in orders ])

# CPO needs lists for pack constraint
order_weights = [ o.weight for o in orders ]

# The heaviest slab
max_slab_weight = max(slab_weights)

# The amount of loss incurred for different amounts of slab use
# The loss will depend on how much less steel is used than the slab
# just large enough to produce the coils.
loss = [ min([sw-use for sw in slab_weights if sw >= use]) for use in range(max_slab_weight+1)]

<a id="Step-4:-Set-up-the-prescriptive-model"></a>
## 4. Set up the prescriptive model

Creates the CPO model.

In [7]:
mdl = CpoModel(name="trucks")

<a id="Define-the-decision-variables"></a>
### 4.1 Define the decision variables

In [8]:
# Which slab is used to produce each coil
production_slab = integer_var_dict(orders, 0, NB_SLABS-1, "production_slab")

# How much of each slab is used
slab_use = integer_var_list(NB_SLABS, 0, max_slab_weight, "slab_use")

<a id="Express-the-business-constraints"></a>
### 4.2 Express the business constraints

In [9]:
# The total loss is
total_loss = sum([element(slab_use[s], loss) for s in slabs])

# The orders are allocated to the slabs with capacity
mdl.add(pack(slab_use, [production_slab[o] for o in orders], order_weights))

# At most MAX_COLOR_PER_SLAB colors per slab
for s in slabs:
   su = 0
   for c in allcolors:
       lo = False
       for o in orders:
           if o.color==c:
               lo = (production_slab[o] == s) | lo
       su += lo
   mdl.add(su <= MAX_COLOR_PER_SLAB)

<a id="Express-objective"></a>
### 4.3 Express the objective

In [10]:
# Add minimization objective
mdl.add(minimize(total_loss))

<a id="Solve-with-Decision-Optimization-solve-service"></a>
### 4.4 Solve with Decision Optimization solve service

In [11]:
print("\nSolving model....")
# Search strategy
mdl.set_search_phases([search_phase([production_slab[o] for o in orders])])

msol = mdl.solve(url=SVC_URL, key=SVC_KEY, FailLimit=100000, TimeLimit=10)


Solving model....


CpoException: Your DOcplexcloud key has not been set

<a id ="Step-5:-Investigate-the-solution-and-then-run-an-example-analysis"></a>
## 5. Investigate the solution and then run an example analysis

#### Print Solution

In [ ]:
if msol:
    print("Solution: ")
    from_slabs = [set([o.index for o in orders if msol[production_slab[o]]== s])for s in slabs]
    slab_colors = [set([o.color for o in orders if o.index in from_slabs[s]])for s in slabs]
    for s in slabs:
        if len(from_slabs[s]) > 0:
            print("Slab = " + str(s))
            print("\tLoss = " + str(loss[msol[slab_use[s]]]))
            print("\tcolors = " + str(slab_colors[s]))
            print("\tOrders = " + str(from_slabs[s]) + "\n")
else:
    print("No solution found")

## Summary

You learned how to set up and use the IBM Decision Optimization CPLEX Modeling for Python to formulate a Constraint Programming model and solve it with IBM Decision Optimization on the cloud.

##  References
* [CPLEX Modeling for Python documentation](https://rawgit.com/IBMDecisionOptimization/docplex-doc/master/docs/index.html)
* [Decision Optimization on Cloud](https://developer.ibm.com/docloud/)
* [Decision Optimization documentation](https://datascience.ibm.com/docs/content/DO/DOinDSX.html)
* For help with DOcplex, or to report a defect, go [here](https://developer.ibm.com/answers/smartspace/docloud).
* Contact us at dofeedback@wwpdl.vnet.ibm.com

<div class="alert alert-block alert-info"> Note: To save resources and get the best performance please use the code below to stop the kernel before exiting your notebook.</div>

In [ ]:
%%javascript
Jupyter.notebook.session.delete();

Copyright © 2017 IBM. IPLA licensed Sample Materials.